<a href="https://colab.research.google.com/github/abcdise/master_thesis/blob/main/MasterThesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf

In [11]:
class EulerMaruyama:

  # ==== Public ==== #

  # Constructor
  def __init__(self, start_time: float, end_time: float, num_intervals: int, \
               diffusion_dimension=1, num_noises=1, num_paths=10000, only_gaussian_noise=True):
    self.__start_time = start_time
    self.__end_time = end_time
    self.__num_intervals = num_intervals
    self.__only_gaussian_noise = only_gaussian_noise
    self.__num_paths = num_paths
    self.__time_intervals = np.linspace(start_time,
                                       end_time,
                                       num=num_intervals+1)
    self.__paths = np.zeros(size=(num_paths, num_intervals+1, diffusion_dimension))

  # Set and get private attributes
  def set_start_time(self, start_time):
    self.__start_time = start_time

  def set_end_time(self, end_time):
    self.__end_time = end_time

  def set_num_intervals(self, num_intervals):
    self.__num_intervals = num_intervals

  def set_time_intervals(self, time_intervals):
    self.__time_intervals = time_intervals

  def set_dimension(self, d):
    self.__dimension = d

  def set_num_paths(self, num_paths):
    self.__num_paths = num_paths

  def get_num_intervals(self):
    return self.__num_intervals

  def get_time_intervals(self):
    return self.__time_intervals

  def get_start_time(self):
    return self.__start_time

  def get_end_time(self):
    return self.__end_time

  def get_dimension(self):
    return self.__dimension

  def getNumPaths(self):
    return self.__num_paths

  def getPaths(self):
    return self.__paths

  # Public methods
  def generate_paths(self, diffusion){
      for i in range(self.__num_paths){
          self.__paths[i] = self.__generate_path(diffusion, self.__noise[i])
      }
  }




  # ==== Private ==== #

  # Private attributes
  __start_time = None
  __end_time = None
  __num_intervals = None
  __time_intervals = None
  __dimension = None
  __num_paths = None
  __num_noises = None
  __noise = None
  __paths = None

  # Private methods
  def __update_noise(self):
    self.__noise = np.random.multivariate_normal(mean=np.zeros(self.__num_noises),
                                                 cov=np.eye(self.__num_noises),
                                                 size=(self.__num_paths, self.__time_intervals))

    for i in range(self.__num_intervals):
      h = self.__time_intervals[i+1] - self.__time_intervals[i]
      self.__noise[:,i,:] *= np.sqrt(h)

  # Generate a path of the diffusion
  def __generate_path(self, diffusion, noise):
    ret = np.zeros(shape=(self.__num_intervals+1, self.__dimension))
    # Set the starting point of the diffusion
    x_0 = diffusion.get_starting_point()
    if x_0.size < self.__dimension:
      ret[0] = x_0 * np.ones(self.__dimension)
    else:
      ret[0] = x_0
    # Simulate the following states
    for i in range(1, self._num_intervals+1):
      h = self.__time_intervals[i] - self.__time_intervals[i-1]
      delta_drift = diffusion.drift(self.__time_intervals[i-1], ret[i-1]) * h
      delta_diffusion = np.matmul(diffusion.diffusion(self.__time[i-1], ret[i-1]),
                                  noise[i])
      ret[i] = ret[i-1] + delta_drift + delta_diffusion
    return ret





In [11]:
class ItoDiffusion:

  # ==== Public ==== #

  # Constructor
  def __init__(self, drift_parameter, diffusion_parameter, starting_point):
    self.drift = drift_parameter
    self.diffusion = diffusion_parameter
    self.__starting_point = starting_point

  def get_starting_point(self):
    return self.__starting_point

  def simulate(self, solver):
    return 0;

  # ==== Private ==== #
  __starting_point = None



In [3]:
mean = np.zeros(2)
cov = np.eye(2)
x = np.random.multivariate_normal(mean, cov, size=(5,10))
x[:, 1, :] * 2

array([[ 2.3033826 ,  1.69429731],
       [ 0.62208391, -0.58793983],
       [ 3.80730913, -2.133019  ],
       [ 0.49997592,  0.08766692],
       [-1.47053935, -0.02997737]])